In [1]:
!pip install google-search-results

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain import SerpAPIWrapper, LLMMathChain
from langchain.llms import OpenAI
from langchain.agents import load_tools, Tool, initialize_agent

llm = OpenAI(temperature=0.7)

tool_names = ["serpapi", "llm-math"]

tools = load_tools(tool_names, llm=llm)


agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [4]:
agent.run("最新のドル円の為替レートを2乗してください")



> Entering new AgentExecutor chain...
 為替レートを検索して調べる
Action: Search
Action Input: 最新のドル円の為替レート
Observation: 21日の東京外国為替市場で円相場は反落した。17時時点は1ドル=145円43～44銭と、前週末の同時点に比べ14銭の円安・ドル高だった。米長期金利が日本時間21日の取引時間 ...
Thought: 調べた値を2乗する
Action: Calculator
Action Input: 145.43^2
Observation: Answer: 21149.8849
Thought: 21149.8849が答えになる
Final Answer: 21149.8849

> Finished chain.


'21149.8849'

In [6]:
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

agent.run("最新のドル円の為替レートを2乗してください")



> Entering new AgentExecutor chain...
 I should use the calculator to solve this question
Action: Calculator
Action Input: Current exchange rate of USD to JPY squared
Observation: Answer: 11427.61
Thought: I now know the final answer
Final Answer: 11427.61

> Finished chain.


'11427.61'

In [7]:
search.run("最新の為替")

'21日の東京外国為替市場で円相場は反落した。17時時点は1ドル=145円43～44銭と、前週末の同時点に比べ14銭の円安・ドル高だった。米長期金利が日本時間21日の取引時間 ...'

In [8]:
llm_math_chain.run("2*2")



> Entering new LLMMathChain chain...
2*2```text
2*2
```
...numexpr.evaluate("2*2")...

Answer: 4
> Finished chain.


'Answer: 4'

In [9]:
print(llm_math_chain.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



In [10]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
